Crawl dữ liệu từ shopee theo từng category

In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pandas as pd

PATH = "C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"


In [2]:
driver = webdriver.Edge(PATH)
# get tạm để lấy cache
driver.get('https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Nokia-6300-%C4%91%E1%BA%A7y-%C4%91%E1%BB%A7-pin-s%E1%BA%A1c-m%C3%A0u-b%E1%BA%A1c-i.39413762.5162298482')

In [3]:
search_list = pd.read_csv('search.csv',sep='\t')
search_list.to_csv('search_withid.csv',sep='\t',index_label='id')
search_list = pd.read_csv('search_withid.csv',sep='\t')
search_list.head()

,id,name,category,n_sold,price,shop_address,image_url,url
0,0,Điện thoại Xiaomi Redmi Note 9S 4GB/64GB - Hàn...,DIENTHOAI,5600.0,5490000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/b982abe468e4813d2ba0...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
1,1,Điện thoại OPPO A37 Fullbox Mới - 2GB/16GB - B...,DIENTHOAI,7800.0,1299000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/654146756efe4b5fe7cf...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
2,2,điện thoại Samsung Galaxy S7 chính hãng / full...,DIENTHOAI,6600.0,1899000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/622be579c39efa0ea92b...,https://shopee.vn/%C4%91i%E1%BB%87n-tho%E1%BA%...
3,3,Điện thoại Nokia 1280 Huyền thoại chính hãng k...,DIENTHOAI,4100.0,189900.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/c92a82f0a1e8076fcc1e...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
4,4,Điện thoại Nokia 1280 Nguyên zin Mới tinh chín...,DIENTHOAI,8800.0,189900.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/c92a82f0a1e8076fcc1e...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...


In [4]:
def reformate_numeric(x):
    if x[-1] == 'k':
        x = float(x.replace(",",".").strip("k")) * 1000
    else:
        x = float(x)
    return x

In [28]:
from cssselector import *
missing_i = []
f = open('items_cmt.csv', 'w', encoding="utf-8")
f.write(f'id\trating\tcontent\n')
for i,search_item in tqdm(search_list.iterrows(),total=len(search_list),position=0, leave=True):
    try:
        curl = search_item.url
        driver.get(curl)
        sleep(1)
        total_height = int(driver.execute_script("return document.body.scrollHeight"))
        for j in range(1, total_height, 100):
            driver.execute_script("window.scrollTo(0, {});".format(j))
            sleep(0.5)
        list_cmt = driver.find_elements_by_css_selector(CSSSelector.CMT_LIST)
        cid = search_item['id']
        for it in list_cmt:
            stars = it.find_element_by_css_selector(CSSSelector.CMT_RATING)
            stars = stars.find_elements_by_css_selector(CSSSelector.CMT_STAR_ACTIVE)
            rating = len(stars)
            content = it.find_element_by_css_selector(CSSSelector.CMT_CONTENT).text.replace('\n','. ')             
            f.write(f'{cid}\t{rating}\t{content}\n')
    except:
        # print('Missing ',i)
        missing_i.append(i)
    #     break
    # if i>=5: break
f.close()

100%|██████████| 100/100 [1:04:25<00:00, 38.65s/it]


In [29]:
missing_i

[]

In [27]:
from cssselector import *
f = open('items_cmt.csv', 'a', encoding="utf-8")
search_list_missing = search_list.iloc[missing_i]
missing_i = []
for i,search_item in tqdm(search_list_missing.iterrows(),total=len(search_list_missing),position=0, leave=True):
    try:
        curl = search_item.url
        driver.get(curl)
        sleep(1)
        total_height = int(driver.execute_script("return document.body.scrollHeight"))
        for j in range(1, total_height, 100):
            driver.execute_script("window.scrollTo(0, {});".format(j))
            sleep(0.5)

        list_cmt = driver.find_elements_by_css_selector(CSSSelector.CMT_LIST)
        cid = search_item['id']
        for it in list_cmt:
            stars = it.find_element_by_css_selector(CSSSelector.CMT_RATING)
            stars = stars.find_elements_by_css_selector(CSSSelector.CMT_STAR_ACTIVE)
            rating = len(stars)
            content = it.find_element_by_css_selector(CSSSelector.CMT_CONTENT).text.replace('\n','. ')   
            f.write(f'{cid}\t{rating}\t{content}\n')
    except:
        # print('Missing ',i)
        missing_i.append(i)
    #     break
    # break
f.close()

0it [00:00, ?it/s]


In [13]:
pd.read_csv('items_cmt.csv', sep='\t').sort_values(by=['id'], ascending=True, axis=0).to_csv('items_cmt_sorted_id.csv',sep='\t',index=False)

# using requests

In [31]:
search_list_full = pd.read_csv('search.csv',sep='\t')
search_list_full.to_csv('search_withid.csv',sep='\t',index_label='id')
search_list_full = pd.read_csv('search_withid.csv',sep='\t')
search_list_full.head()

,id,name,category,n_sold,price,shop_address,image_url,url
0,0,Điện thoại OPPO A37 Fullbox Mới - 2GB/16GB - B...,DIENTHOAI,7800.0,1299000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/654146756efe4b5fe7cf...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
1,1,điện thoại Samsung Galaxy S7 chính hãng / full...,DIENTHOAI,6600.0,1899000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/622be579c39efa0ea92b...,https://shopee.vn/%C4%91i%E1%BB%87n-tho%E1%BA%...
2,2,Điện thoại Nokia 1280 Nguyên zin Mới tinh chín...,DIENTHOAI,8800.0,189900.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/c92a82f0a1e8076fcc1e...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
3,3,ĐIỆN THOẠI NOKIA 1202 MỚI TINH FULL SẠC PIN CH...,DIENTHOAI,6100.0,175000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/b36a13ed46c0e3feab04...,https://shopee.vn/%C4%90I%E1%BB%86N-THO%E1%BA%...
4,4,điện thoại Samsung Galaxy S7 Edge 2 sim ram 4G...,DIENTHOAI,5400.0,2799000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/deb114c783e8feb58d6c...,https://shopee.vn/%C4%91i%E1%BB%87n-tho%E1%BA%...


In [26]:
import requests
# r = requests.get('https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid=4952265828&limit=20&offset=0&shopid=203317301&type=0')
# for ob in r.json()['data']['ratings']:
#     print(ob['rating_star'], ob['comment'])

missing_i = []
f = open('items_cmt.csv', 'w', encoding="utf-8")
f.write(f'id\trating\tcontent\n')
for i,search_item in tqdm(search_list_full.iterrows(),total=len(search_list_full),position=0, leave=True):
    try:
        curl = search_item.url
        curl_split = curl.split('.')
        shopid = curl_split[-2]
        itemid = curl_split[-1]
        r = requests.get(f'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid={itemid}&limit=6&offset=0&shopid={shopid}&type=0')
        cid = search_item['id']
        for ob in r.json()['data']['ratings']:
            rating = ob['rating_star']
            content = ob['comment'].replace('\n','. ')     
            f.write(f'{cid}\t{rating}\t{content}\n')
    except:
        # print('Missing ',i)
        missing_i.append(i)
    #     break
    # if i>=5: break
f.close()

100%|██████████| 100/100 [00:47<00:00,  2.08it/s]
